In [1]:
# Installing psycopg2 shall fail so use psycopg2-binary
!pip install psycopg2-binary

In [2]:
# Important imports
import psycopg2 as pg
import pandas as pd
import numpy as np

In [3]:
# Establish connection with the database by providing credentials and database
# information.
pg_conn = pg.connect(
    database="",
    user="",
    password="",
    host="",
    port=""
)

pg_conn

<connection object at 0x7f4b0c4842b8; dsn: 'user=livesafe2019 password=xxx dbname=LiveSafeDB host=livesafe-lab12-project.cyplld0jesr7.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [4]:
# Get the data from github link DMA uploaded and check for data consistency
df = pd.read_csv("https://raw.githubusercontent.com/labs12-should-i-live-here/DS/daniel-martinalarcon/data_output/5__NOAA/noaa_1.csv")

In [5]:
# Check the first few rows
df.head()

,year,FIPS,Winter Weather,Storm,Flood,Fire,Heat,Drought,Tornado,Hurricane
0,1996,1001,4,7,2,0,1,0,0,0
1,1996,1003,5,19,2,0,1,0,0,0
2,1996,1005,5,2,0,0,1,0,0,0
3,1996,1007,5,6,2,0,1,0,0,0
4,1996,1009,5,17,0,0,1,0,0,0


In [6]:
# Check for any misisng values
df.isna().sum()

year              0
FIPS              0
Winter Weather    0
Storm             0
Flood             0
Fire              0
Heat              0
Drought           0
Tornado           0
Hurricane         0
dtype: int64

In [7]:
# Check the shape
df.shape

(72243, 10)

In [8]:
# Check the data types
df.dtypes

year              int64
FIPS              int64
Winter Weather    int64
Storm             int64
Flood             int64
Fire              int64
Heat              int64
Drought           int64
Tornado           int64
Hurricane         int64
dtype: object

In [9]:
# Define the template of the table operation.
drop_noaa_table = """DROP TABLE IF EXISTS noaa_historical;"""

# no primary key assigned since year or/and fipscode both
# repeat.
create_noaa_table = """ CREATE TABLE noaa_historical(
    year int,
    fipscode int,
    winterweather int,
    storm int,
    flood int,
    fire int,
    heat int,
    drought int,
    tornado int,
    hurricane int
);"""

# Get count of number of records in the table
get_count_noaa_table = """SELECT count(*) FROM noaa_historical;"""

In [10]:
# Create the table and commit
pg_cur = pg_conn.cursor()
pg_cur.execute(drop_noaa_table)
pg_cur.execute(create_noaa_table)
pg_conn.commit()

In [11]:
pg_cur.execute(get_count_noaa_table)
count = pg_cur.fetchone()
print(count)

(0,)


In [12]:
# Get the csv file locally for loading
!wget https://raw.githubusercontent.com/labs12-should-i-live-here/DS/daniel-martinalarcon/data_output/5__NOAA/noaa_1.csv

--2019-05-02 14:42:00--  https://raw.githubusercontent.com/labs12-should-i-live-here/DS/daniel-martinalarcon/data_output/5__NOAA/noaa_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.208.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.208.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1974532 (1.9M) [text/plain]
Saving to: ‘noaa_1.csv’

noaa_1.csv          100%[===================>]   1.88M  9.12MB/s    in 0.2s    

2019-05-02 14:42:00 (9.12 MB/s) - ‘noaa_1.csv’ saved [1974532/1974532]



In [13]:
with open('noaa_1.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f)  # Skip the header row.
    pg_cur.copy_from(f, 'noaa_historical', sep=',')

pg_conn.commit()

In [14]:
# Check the row count in noaa_historical after insert
pg_cur.execute(get_count_noaa_table)
count = pg_cur.fetchone()
print(count)

(72243,)


In [15]:
# Sample select testing 1
get_all_records_in_2000 = """ SELECT * FROM noaa_historical
WHERE year=2000;
"""
pg_cur.execute(get_all_records_in_2000)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2000, 1001, 1, 16, 1, 0, 0, 0, 1, 0)
(2000, 1003, 1, 18, 0, 0, 0, 0, 4, 0)
(2000, 1005, 1, 3, 0, 0, 0, 0, 0, 0)
(2000, 1007, 1, 10, 0, 0, 0, 0, 1, 0)
(2000, 1009, 2, 9, 1, 0, 0, 0, 0, 0)
(2000, 1011, 1, 2, 0, 0, 0, 0, 3, 0)
(2000, 1013, 1, 5, 0, 0, 0, 0, 0, 0)
(2000, 1015, 1, 6, 0, 0, 0, 0, 2, 0)
(2000, 1017, 1, 2, 0, 0, 0, 0, 0, 0)
(2000, 1019, 1, 4, 0, 0, 0, 0, 1, 0)
(2000, 1021, 1, 8, 0, 0, 0, 0, 1, 0)
(2000, 1023, 1, 14, 1, 0, 0, 0, 1, 0)
(2000, 1025, 1, 13, 0, 0, 0, 0, 0, 0)
(2000, 1027, 1, 10, 0, 0, 0, 0, 0, 0)
(2000, 1029, 1, 6, 0, 0, 0, 0, 1, 0)
(2000, 1031, 1, 9, 0, 0, 0, 0, 1, 0)
(2000, 1033, 1, 15, 0, 0, 0, 0, 0, 0)
(2000, 1035, 1, 7, 0, 0, 0, 0, 0, 0)
(2000, 1037, 1, 3, 0, 0, 0, 0, 1, 0)
(2000, 1039, 2, 16, 0, 0, 1, 0, 0, 0)
(2000, 1041, 1, 5, 0, 0, 0, 0, 0, 0)
(2000, 1043, 1, 28, 2, 0, 0, 0, 0, 0)
(2000, 1045, 0, 7, 0, 0, 0, 0, 1, 0)
(2000, 1047, 0, 8, 0, 0, 0, 0, 0, 0)
(2000, 1049, 0, 16, 0, 0, 0, 0, 0, 0)
(2000, 1051, 2, 11, 1, 0, 1, 0, 0, 0)
(2000, 1053, 2, 6, 0, 0, 1,

(2000, 26109, 0, 4, 0, 0, 0, 0, 0, 0)
(2000, 26111, 0, 4, 0, 0, 0, 0, 0, 0)
(2000, 26113, 0, 2, 0, 0, 0, 0, 0, 0)
(2000, 26115, 0, 6, 1, 0, 0, 0, 0, 0)
(2000, 26117, 0, 4, 1, 0, 0, 0, 0, 0)
(2000, 26119, 0, 3, 0, 0, 0, 0, 0, 0)
(2000, 26121, 0, 5, 1, 0, 0, 0, 0, 0)
(2000, 26123, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 26125, 0, 35, 3, 0, 0, 0, 0, 0)
(2000, 26127, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 26129, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 26131, 0, 2, 0, 1, 0, 0, 0, 0)
(2000, 26133, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 26135, 0, 4, 0, 0, 0, 0, 0, 0)
(2000, 26137, 0, 1, 0, 0, 0, 0, 0, 0)
(2000, 26139, 0, 3, 1, 0, 0, 0, 0, 0)
(2000, 26141, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 26143, 0, 2, 0, 0, 0, 0, 0, 0)
(2000, 26145, 0, 4, 0, 0, 0, 0, 0, 0)
(2000, 26147, 0, 14, 0, 0, 0, 0, 0, 0)
(2000, 26149, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 26151, 0, 7, 1, 0, 0, 0, 0, 0)
(2000, 26153, 0, 3, 0, 0, 0, 0, 0, 0)
(2000, 26155, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 26157, 0, 8, 0, 0, 0, 0, 0, 0)
(2000, 26159, 0, 8, 1, 0, 0, 0, 1, 0)
(2000, 261

(2000, 34023, 6, 21, 8, 2, 3, 1, 0, 0)
(2000, 34025, 7, 24, 10, 1, 3, 1, 0, 0)
(2000, 34027, 0, 14, 3, 0, 0, 0, 0, 0)
(2000, 34029, 0, 10, 1, 0, 0, 0, 0, 0)
(2000, 34031, 0, 9, 0, 0, 0, 0, 0, 0)
(2000, 34033, 0, 6, 0, 0, 0, 0, 0, 0)
(2000, 34035, 0, 14, 4, 0, 0, 0, 0, 0)
(2000, 34037, 0, 9, 3, 0, 0, 0, 0, 0)
(2000, 34039, 0, 7, 2, 0, 0, 0, 0, 0)
(2000, 34041, 0, 7, 7, 0, 0, 0, 0, 0)
(2000, 35001, 0, 8, 0, 0, 2, 7, 0, 0)
(2000, 35003, 1, 3, 0, 0, 2, 7, 0, 0)
(2000, 35005, 3, 8, 1, 0, 2, 7, 0, 0)
(2000, 35006, 2, 1, 0, 0, 2, 7, 0, 0)
(2000, 35007, 1, 4, 0, 0, 2, 7, 0, 0)
(2000, 35009, 1, 8, 2, 0, 2, 7, 2, 0)
(2000, 35011, 2, 9, 0, 0, 2, 7, 0, 0)
(2000, 35013, 2, 3, 1, 0, 2, 7, 0, 0)
(2000, 35015, 0, 11, 9, 0, 2, 7, 0, 0)
(2000, 35017, 3, 4, 1, 0, 2, 6, 0, 0)
(2000, 35019, 2, 4, 2, 0, 2, 7, 0, 0)
(2000, 35021, 2, 0, 1, 0, 2, 7, 0, 0)
(2000, 35023, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 35025, 1, 4, 2, 0, 0, 0, 0, 0)
(2000, 35027, 0, 2, 1, 1, 0, 0, 0, 0)
(2000, 35028, 0, 3, 4, 1, 0, 0, 0, 0)
(2000

(2000, 42081, 0, 18, 1, 0, 0, 0, 1, 0)
(2000, 42083, 0, 10, 1, 0, 0, 0, 0, 0)
(2000, 42085, 0, 12, 2, 0, 0, 0, 0, 0)
(2000, 42087, 0, 2, 0, 0, 0, 0, 0, 0)
(2000, 42089, 0, 11, 1, 0, 0, 0, 0, 0)
(2000, 42091, 0, 7, 3, 0, 0, 0, 0, 0)
(2000, 42093, 0, 3, 1, 0, 0, 0, 0, 0)
(2000, 42095, 0, 17, 4, 3, 0, 0, 0, 0)
(2000, 42097, 0, 6, 1, 0, 0, 0, 0, 0)
(2000, 42099, 0, 4, 1, 0, 0, 0, 0, 0)
(2000, 42101, 0, 6, 2, 0, 0, 0, 0, 0)
(2000, 42103, 0, 5, 1, 0, 0, 0, 0, 0)
(2000, 42105, 0, 7, 0, 0, 0, 0, 0, 0)
(2000, 42107, 0, 10, 1, 0, 0, 0, 0, 0)
(2000, 42109, 0, 4, 0, 0, 0, 0, 0, 0)
(2000, 42111, 0, 3, 4, 0, 0, 0, 1, 0)
(2000, 42113, 0, 7, 1, 0, 0, 0, 1, 0)
(2000, 42115, 0, 8, 0, 0, 0, 0, 0, 0)
(2000, 42117, 0, 9, 2, 0, 0, 0, 0, 0)
(2000, 42119, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 42121, 0, 13, 1, 0, 0, 0, 0, 0)
(2000, 42123, 0, 10, 0, 0, 0, 0, 0, 0)
(2000, 42125, 0, 5, 6, 0, 0, 0, 0, 0)
(2000, 42127, 0, 7, 3, 0, 0, 0, 0, 0)
(2000, 42129, 0, 20, 6, 0, 0, 0, 0, 0)
(2000, 42131, 0, 3, 1, 0, 0, 0, 0, 0)
(20

(2000, 49015, 1, 0, 0, 0, 0, 0, 0, 0)
(2000, 49017, 4, 2, 0, 0, 0, 0, 1, 0)
(2000, 49019, 0, 7, 2, 4, 0, 0, 2, 0)
(2000, 49021, 0, 0, 1, 0, 0, 0, 0, 0)
(2000, 49023, 4, 2, 0, 0, 0, 0, 0, 0)
(2000, 49025, 1, 2, 0, 0, 0, 0, 0, 0)
(2000, 49027, 1, 2, 0, 0, 0, 0, 0, 0)
(2000, 49029, 1, 1, 1, 0, 0, 0, 0, 0)
(2000, 49031, 0, 1, 0, 0, 0, 0, 0, 0)
(2000, 49033, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 49035, 0, 8, 0, 0, 0, 0, 1, 0)
(2000, 49037, 0, 3, 0, 7, 0, 0, 0, 0)
(2000, 49039, 0, 1, 0, 0, 0, 0, 1, 0)
(2000, 49041, 0, 0, 1, 0, 0, 0, 0, 0)
(2000, 49043, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 49045, 0, 5, 2, 0, 0, 0, 0, 0)
(2000, 49047, 0, 3, 0, 0, 0, 0, 0, 0)
(2000, 49049, 0, 4, 1, 0, 0, 0, 0, 0)
(2000, 49051, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 49053, 0, 2, 3, 0, 0, 0, 0, 0)
(2000, 49055, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 49057, 0, 0, 1, 0, 0, 0, 0, 0)
(2000, 50001, 11, 2, 4, 0, 0, 0, 0, 0)
(2000, 50003, 18, 5, 4, 0, 0, 0, 0, 0)
(2000, 50005, 14, 8, 4, 0, 0, 0, 0, 0)
(2000, 50007, 20, 11, 2, 0, 0, 0, 0, 0)
(2000, 

In [16]:
# Sample select testing 2
get_all_records_between_2000_2005 = """ SELECT * FROM noaa_historical
WHERE year BETWEEN 2000 AND 2005;
"""
pg_cur.execute(get_all_records_between_2000_2005)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2000, 1001, 1, 16, 1, 0, 0, 0, 1, 0)
(2000, 1003, 1, 18, 0, 0, 0, 0, 4, 0)
(2000, 1005, 1, 3, 0, 0, 0, 0, 0, 0)
(2000, 1007, 1, 10, 0, 0, 0, 0, 1, 0)
(2000, 1009, 2, 9, 1, 0, 0, 0, 0, 0)
(2000, 1011, 1, 2, 0, 0, 0, 0, 3, 0)
(2000, 1013, 1, 5, 0, 0, 0, 0, 0, 0)
(2000, 1015, 1, 6, 0, 0, 0, 0, 2, 0)
(2000, 1017, 1, 2, 0, 0, 0, 0, 0, 0)
(2000, 1019, 1, 4, 0, 0, 0, 0, 1, 0)
(2000, 1021, 1, 8, 0, 0, 0, 0, 1, 0)
(2000, 1023, 1, 14, 1, 0, 0, 0, 1, 0)
(2000, 1025, 1, 13, 0, 0, 0, 0, 0, 0)
(2000, 1027, 1, 10, 0, 0, 0, 0, 0, 0)
(2000, 1029, 1, 6, 0, 0, 0, 0, 1, 0)
(2000, 1031, 1, 9, 0, 0, 0, 0, 1, 0)
(2000, 1033, 1, 15, 0, 0, 0, 0, 0, 0)
(2000, 1035, 1, 7, 0, 0, 0, 0, 0, 0)
(2000, 1037, 1, 3, 0, 0, 0, 0, 1, 0)
(2000, 1039, 2, 16, 0, 0, 1, 0, 0, 0)
(2000, 1041, 1, 5, 0, 0, 0, 0, 0, 0)
(2000, 1043, 1, 28, 2, 0, 0, 0, 0, 0)
(2000, 1045, 0, 7, 0, 0, 0, 0, 1, 0)
(2000, 1047, 0, 8, 0, 0, 0, 0, 0, 0)
(2000, 1049, 0, 16, 0, 0, 0, 0, 0, 0)
(2000, 1051, 2, 11, 1, 0, 1, 0, 0, 0)
(2000, 1053, 2, 6, 0, 0, 1,

(2000, 12013, 0, 6, 0, 0, 0, 0, 0, 0)
(2000, 12015, 0, 6, 1, 2, 0, 0, 1, 1)
(2000, 12017, 0, 10, 3, 2, 0, 0, 0, 1)
(2000, 12019, 0, 24, 0, 0, 0, 0, 0, 1)
(2000, 12021, 0, 5, 3, 1, 0, 0, 0, 0)
(2000, 12023, 0, 17, 1, 0, 0, 0, 0, 0)
(2000, 12027, 0, 3, 1, 1, 0, 0, 0, 1)
(2000, 12029, 0, 2, 0, 0, 0, 0, 0, 1)
(2000, 12031, 0, 19, 2, 0, 0, 0, 2, 0)
(2000, 12033, 0, 14, 1, 1, 0, 0, 1, 0)
(2000, 12035, 0, 6, 0, 1, 0, 0, 0, 0)
(2000, 12037, 0, 1, 1, 1, 0, 0, 5, 0)
(2000, 12039, 14, 4, 0, 0, 0, 0, 1, 1)
(2000, 12041, 0, 2, 0, 0, 0, 0, 0, 0)
(2000, 12043, 13, 1, 0, 2, 0, 0, 0, 0)
(2000, 12045, 0, 6, 1, 3, 0, 0, 1, 0)
(2000, 12047, 0, 12, 1, 0, 0, 0, 0, 0)
(2000, 12049, 7, 2, 2, 0, 0, 0, 0, 1)
(2000, 12051, 6, 4, 1, 1, 0, 0, 1, 1)
(2000, 12053, 0, 7, 0, 1, 0, 0, 1, 0)
(2000, 12055, 4, 5, 2, 1, 0, 0, 1, 1)
(2000, 12057, 3, 42, 9, 1, 0, 0, 2, 1)
(2000, 12059, 0, 7, 0, 0, 0, 0, 1, 0)
(2000, 12061, 5, 2, 1, 0, 0, 0, 0, 1)
(2000, 12063, 2, 18, 1, 0, 0, 0, 2, 0)
(2000, 12065, 1, 5, 2, 0, 0, 0, 2, 1)
(2

(2000, 30077, 0, 1, 0, 0, 0, 0, 0, 0)
(2000, 30079, 0, 18, 1, 0, 0, 0, 0, 0)
(2000, 30081, 0, 1, 0, 1, 0, 0, 0, 0)
(2000, 30083, 0, 9, 0, 0, 0, 0, 0, 0)
(2000, 30085, 0, 14, 1, 0, 0, 0, 0, 0)
(2000, 30087, 0, 22, 0, 0, 0, 0, 0, 0)
(2000, 30089, 0, 0, 0, 0, 0, 0, 0, 0)
(2000, 30091, 0, 11, 1, 0, 0, 0, 0, 0)
(2000, 30093, 0, 2, 0, 0, 0, 0, 0, 0)
(2000, 30095, 0, 2, 0, 0, 0, 0, 0, 0)
(2000, 30097, 0, 3, 0, 0, 0, 0, 0, 0)
(2000, 30099, 0, 5, 0, 4, 0, 0, 0, 0)
(2000, 30101, 0, 5, 0, 0, 0, 0, 0, 0)
(2000, 30103, 0, 4, 0, 0, 0, 0, 0, 0)
(2000, 30105, 0, 57, 2, 0, 0, 0, 4, 0)
(2000, 30107, 0, 6, 0, 0, 0, 0, 0, 0)
(2000, 30109, 0, 9, 0, 0, 0, 0, 0, 0)
(2000, 30111, 0, 20, 0, 0, 0, 0, 0, 0)
(2000, 31001, 3, 11, 1, 0, 0, 0, 1, 0)
(2000, 31003, 2, 21, 2, 0, 0, 0, 3, 0)
(2000, 31005, 1, 9, 0, 0, 0, 0, 0, 0)
(2000, 31007, 1, 4, 0, 0, 0, 0, 1, 0)
(2000, 31009, 1, 13, 0, 0, 0, 0, 0, 0)
(2000, 31011, 4, 9, 2, 0, 0, 0, 0, 0)
(2000, 31013, 4, 5, 0, 0, 0, 3, 0, 0)
(2000, 31015, 3, 2, 0, 0, 0, 0, 0, 0)
(20

(2000, 53075, 0, 2, 1, 0, 0, 0, 0, 0)
(2000, 53077, 0, 1, 0, 2, 0, 0, 1, 0)
(2000, 54001, 1, 6, 1, 0, 0, 0, 0, 0)
(2000, 54003, 1, 12, 0, 0, 0, 0, 0, 0)
(2000, 54005, 5, 14, 3, 1, 4, 0, 0, 0)
(2000, 54007, 4, 10, 3, 0, 4, 0, 0, 0)
(2000, 54009, 5, 7, 2, 0, 4, 0, 0, 0)
(2000, 54011, 6, 8, 3, 0, 4, 0, 1, 0)
(2000, 54013, 4, 9, 2, 0, 4, 0, 0, 0)
(2000, 54015, 4, 9, 0, 0, 4, 0, 0, 0)
(2000, 54017, 6, 6, 1, 0, 4, 0, 0, 0)
(2000, 54019, 6, 12, 1, 0, 4, 0, 0, 0)
(2000, 54021, 1, 3, 1, 0, 0, 0, 0, 0)
(2000, 54023, 6, 6, 2, 0, 0, 0, 0, 0)
(2000, 54025, 5, 12, 1, 0, 4, 0, 0, 0)
(2000, 54027, 5, 16, 1, 0, 4, 0, 0, 0)
(2000, 54029, 6, 7, 1, 0, 4, 0, 0, 0)
(2000, 54031, 7, 15, 3, 0, 4, 0, 0, 0)
(2000, 54033, 8, 10, 3, 0, 3, 0, 0, 0)
(2000, 54035, 9, 11, 2, 0, 4, 0, 0, 0)
(2000, 54037, 12, 22, 1, 0, 4, 0, 0, 0)
(2000, 54039, 11, 26, 5, 0, 4, 0, 0, 0)
(2000, 54041, 5, 6, 3, 0, 0, 0, 0, 0)
(2000, 54043, 1, 7, 0, 0, 0, 0, 1, 0)
(2000, 54045, 5, 6, 1, 1, 0, 0, 0, 0)
(2000, 54047, 16, 8, 2, 1, 4, 0, 0, 0

(2001, 29023, 3, 4, 9, 0, 4, 0, 0, 0)
(2001, 29025, 2, 8, 0, 0, 4, 0, 0, 0)
(2001, 29027, 0, 6, 4, 0, 7, 0, 1, 0)
(2001, 29029, 3, 21, 11, 0, 4, 0, 0, 0)
(2001, 29031, 2, 10, 10, 0, 4, 0, 0, 0)
(2001, 29033, 2, 6, 0, 0, 4, 0, 0, 0)
(2001, 29035, 0, 6, 2, 0, 7, 0, 0, 0)
(2001, 29037, 3, 28, 7, 0, 4, 0, 1, 0)
(2001, 29039, 2, 9, 15, 0, 4, 0, 1, 0)
(2001, 29041, 0, 13, 1, 0, 7, 0, 0, 0)
(2001, 29043, 3, 23, 3, 0, 4, 0, 0, 0)
(2001, 29045, 2, 7, 3, 0, 4, 0, 0, 0)
(2001, 29047, 0, 18, 4, 0, 7, 0, 0, 0)
(2001, 29049, 0, 11, 1, 0, 7, 0, 0, 0)
(2001, 29051, 0, 3, 2, 0, 7, 0, 0, 0)
(2001, 29053, 2, 9, 1, 0, 4, 0, 1, 0)
(2001, 29055, 3, 2, 1, 0, 2, 0, 0, 0)
(2001, 29057, 2, 17, 3, 0, 2, 0, 0, 0)
(2001, 29059, 0, 11, 1, 0, 7, 0, 0, 0)
(2001, 29061, 1, 11, 5, 0, 7, 0, 0, 0)
(2001, 29063, 2, 10, 3, 0, 8, 0, 0, 0)
(2001, 29065, 1, 3, 2, 0, 7, 0, 0, 0)
(2001, 29067, 3, 12, 2, 0, 2, 0, 0, 0)
(2001, 29069, 2, 6, 2, 0, 2, 0, 0, 0)
(2001, 29071, 2, 22, 0, 0, 2, 0, 0, 0)
(2001, 29073, 1, 8, 0, 0, 7, 0, 0,

...

(2004, 16033, 0, 3, 0, 0, 0, 0, 0, 0)
(2004, 16035, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16037, 0, 2, 0, 0, 0, 0, 0, 0)
(2004, 16039, 0, 2, 2, 0, 0, 0, 0, 0)
(2004, 16041, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16043, 0, 2, 1, 0, 0, 0, 1, 0)
(2004, 16045, 0, 0, 0, 0, 0, 0, 1, 0)
(2004, 16047, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16049, 0, 8, 0, 0, 0, 0, 0, 0)
(2004, 16051, 0, 7, 0, 0, 0, 0, 0, 0)
(2004, 16053, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16055, 0, 1, 0, 0, 0, 0, 0, 0)
(2004, 16057, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16059, 0, 5, 0, 0, 0, 0, 0, 0)
(2004, 16061, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16063, 0, 1, 0, 0, 0, 0, 0, 0)
(2004, 16065, 0, 2, 1, 0, 0, 0, 0, 0)
(2004, 16067, 0, 4, 0, 0, 0, 0, 0, 0)
(2004, 16069, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16071, 0, 0, 3, 0, 0, 0, 0, 0)
(2004, 16073, 0, 1, 3, 0, 0, 0, 0, 0)
(2004, 16075, 0, 0, 0, 0, 0, 0, 1, 0)
(2004, 16077, 0, 4, 1, 0, 0, 0, 0, 0)
(2004, 16079, 0, 0, 0, 0, 0, 0, 0, 0)
(2004, 16081, 0, 2, 0, 0, 0, 0, 0, 0)
(2004, 16083, 0, 1, 0, 0, 0, 0, 0, 0)
(2004, 16085

(2004, 29169, 0, 18, 0, 0, 0, 0, 0, 0)
(2004, 29171, 0, 6, 4, 0, 0, 0, 0, 0)
(2004, 29173, 0, 7, 3, 0, 0, 0, 1, 0)
(2004, 29175, 0, 6, 1, 0, 0, 0, 0, 0)
(2004, 29177, 0, 6, 1, 0, 0, 0, 0, 0)
(2004, 29179, 0, 4, 0, 0, 0, 0, 2, 0)
(2004, 29181, 0, 5, 3, 0, 0, 0, 0, 0)
(2004, 29183, 0, 25, 2, 0, 0, 0, 2, 0)
(2004, 29185, 0, 5, 0, 0, 0, 0, 1, 0)
(2004, 29186, 0, 11, 0, 0, 0, 0, 1, 0)
(2004, 29187, 0, 13, 0, 0, 0, 0, 2, 0)
(2004, 29189, 0, 78, 4, 0, 0, 0, 1, 0)
(2004, 29195, 0, 11, 1, 0, 0, 0, 1, 0)
(2004, 29197, 0, 1, 0, 0, 0, 0, 0, 0)
(2004, 29199, 0, 1, 0, 0, 0, 0, 0, 0)
(2004, 29201, 0, 4, 0, 0, 0, 0, 2, 0)
(2004, 29203, 0, 5, 0, 0, 0, 0, 1, 0)
(2004, 29205, 0, 4, 1, 0, 0, 0, 0, 0)
(2004, 29207, 0, 9, 1, 0, 0, 0, 0, 0)
(2004, 29209, 0, 27, 3, 0, 0, 0, 0, 0)
(2004, 29211, 0, 6, 2, 0, 0, 0, 0, 0)
(2004, 29213, 0, 12, 2, 0, 0, 0, 0, 0)
(2004, 29215, 0, 7, 0, 0, 0, 0, 0, 0)
(2004, 29217, 0, 9, 0, 0, 0, 0, 0, 0)
(2004, 29219, 0, 4, 1, 0, 0, 0, 0, 0)
(2004, 29221, 0, 16, 0, 0, 0, 0, 5, 0)
(20

(2005, 13079, 2, 6, 3, 0, 0, 0, 0, 2)
(2005, 13081, 1, 9, 2, 0, 0, 0, 0, 2)
(2005, 13083, 2, 7, 1, 0, 0, 0, 0, 2)
(2005, 13085, 1, 11, 0, 0, 0, 0, 0, 2)
(2005, 13087, 0, 5, 1, 0, 1, 0, 1, 0)
(2005, 13089, 2, 17, 6, 0, 0, 0, 0, 2)
(2005, 13091, 1, 9, 3, 0, 0, 0, 0, 2)
(2005, 13093, 1, 14, 3, 0, 0, 0, 0, 2)
(2005, 13095, 1, 2, 1, 0, 0, 0, 0, 2)
(2005, 13097, 1, 8, 2, 0, 0, 0, 0, 2)
(2005, 13099, 0, 2, 0, 0, 1, 0, 1, 0)
(2005, 13101, 0, 1, 0, 0, 1, 0, 0, 0)
(2005, 13103, 1, 7, 2, 0, 0, 0, 0, 2)
(2005, 13105, 1, 11, 3, 0, 0, 0, 0, 2)
(2005, 13107, 1, 6, 2, 0, 0, 0, 0, 2)
(2005, 13109, 1, 5, 1, 0, 0, 0, 1, 2)
(2005, 13111, 1, 6, 2, 0, 0, 0, 1, 2)
(2005, 13113, 1, 22, 2, 0, 0, 0, 1, 2)
(2005, 13115, 0, 15, 0, 0, 1, 0, 0, 0)
(2005, 13117, 0, 10, 0, 0, 1, 0, 1, 0)
(2005, 13119, 0, 8, 1, 0, 1, 0, 0, 0)
(2005, 13121, 0, 18, 4, 0, 0, 0, 0, 1)
(2005, 13123, 0, 5, 0, 0, 0, 0, 0, 1)
(2005, 13125, 0, 2, 0, 0, 0, 0, 0, 1)
(2005, 13127, 0, 6, 1, 0, 0, 0, 1, 1)
(2005, 13129, 0, 5, 2, 0, 0, 0, 0, 1)
(200

(2005, 28049, 0, 39, 3, 0, 2, 0, 5, 2)
(2005, 28051, 0, 11, 2, 0, 2, 0, 2, 2)
(2005, 28053, 0, 3, 0, 0, 2, 0, 4, 2)
(2005, 28055, 0, 8, 0, 0, 2, 0, 0, 2)
(2005, 28057, 0, 3, 1, 0, 2, 0, 0, 2)
(2005, 28059, 0, 7, 2, 0, 1, 0, 0, 2)
(2005, 28061, 0, 29, 1, 0, 1, 0, 2, 2)
(2005, 28063, 0, 9, 1, 0, 1, 0, 7, 2)
(2005, 28065, 0, 19, 3, 0, 2, 0, 1, 1)
(2005, 28067, 0, 24, 4, 0, 0, 0, 1, 2)
(2005, 28069, 0, 16, 1, 0, 0, 0, 2, 1)
(2005, 28071, 0, 4, 0, 0, 0, 0, 0, 1)
(2005, 28073, 0, 12, 3, 0, 1, 0, 1, 1)
(2005, 28075, 0, 23, 1, 0, 0, 0, 3, 2)
(2005, 28077, 0, 22, 1, 0, 0, 0, 1, 1)
(2005, 28079, 0, 6, 2, 0, 0, 0, 4, 2)
(2005, 28081, 0, 10, 5, 0, 0, 0, 0, 1)
(2005, 28083, 0, 14, 1, 0, 0, 0, 6, 0)
(2005, 28085, 0, 33, 1, 0, 0, 0, 3, 0)
(2005, 28087, 0, 19, 1, 0, 0, 0, 0, 0)
(2005, 28089, 0, 30, 2, 0, 0, 0, 2, 0)
(2005, 28091, 0, 13, 3, 0, 0, 0, 1, 0)
(2005, 28093, 0, 5, 0, 0, 0, 0, 0, 0)
(2005, 28095, 0, 3, 3, 0, 0, 0, 2, 0)
(2005, 28097, 0, 4, 0, 0, 0, 0, 0, 0)
(2005, 28099, 0, 14, 2, 0, 0, 0, 6,

(2005, 46127, 0, 23, 3, 0, 0, 0, 1, 0)
(2005, 46129, 0, 7, 0, 0, 0, 0, 0, 0)
(2005, 46135, 0, 6, 1, 0, 0, 0, 0, 0)
(2005, 46137, 0, 30, 0, 0, 0, 0, 0, 0)
(2005, 47001, 0, 15, 0, 0, 0, 0, 0, 0)
(2005, 47003, 0, 4, 1, 0, 0, 0, 0, 0)
(2005, 47005, 0, 7, 0, 0, 0, 0, 3, 0)
(2005, 47007, 0, 13, 1, 0, 0, 0, 0, 0)
(2005, 47009, 0, 20, 0, 0, 0, 0, 0, 0)
(2005, 47011, 0, 18, 1, 0, 0, 0, 0, 0)
(2005, 47013, 1, 10, 1, 0, 0, 0, 0, 0)
(2005, 47015, 1, 2, 0, 0, 0, 0, 0, 0)
(2005, 47017, 1, 5, 0, 0, 0, 0, 0, 0)
(2005, 47019, 0, 8, 0, 0, 0, 0, 0, 0)
(2005, 47021, 0, 5, 0, 0, 0, 0, 0, 0)
(2005, 47023, 0, 5, 0, 0, 0, 0, 0, 0)
(2005, 47025, 0, 10, 0, 0, 0, 0, 0, 0)
(2005, 47027, 0, 6, 0, 0, 0, 0, 0, 0)
(2005, 47029, 0, 9, 0, 0, 0, 0, 0, 0)
(2005, 47031, 0, 9, 0, 0, 0, 0, 0, 0)
(2005, 47033, 0, 6, 0, 0, 0, 0, 1, 0)
(2005, 47035, 1, 7, 0, 0, 0, 0, 0, 0)
(2005, 47037, 1, 25, 1, 0, 0, 0, 1, 0)
(2005, 47039, 1, 3, 0, 0, 0, 0, 0, 0)
(2005, 47041, 4, 3, 0, 0, 0, 0, 0, 0)
(2005, 47043, 4, 9, 2, 0, 0, 0, 2, 0)
(20

In [17]:
# Sample select testing 3
get_all_records_in_county_year_range = """ SELECT * FROM noaa_historical
WHERE fipscode=17033 AND year BETWEEN 2000 AND 2005;
"""
pg_cur.execute(get_all_records_in_county_year_range)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2000, 17033, 3, 5, 3, 0, 0, 0, 0, 0)
(2001, 17033, 0, 8, 2, 0, 0, 0, 0, 0)
(2002, 17033, 1, 8, 3, 0, 0, 0, 0, 0)
(2003, 17033, 2, 9, 0, 0, 0, 0, 0, 0)
(2004, 17033, 2, 8, 3, 0, 0, 0, 0, 0)
(2005, 17033, 1, 2, 2, 0, 0, 1, 0, 0)


In [18]:
# Sample select testing 4
get_all_records_in_county_specific_year = """ SELECT * FROM noaa_historical
WHERE fipscode=17033 AND year=2005;
"""
pg_cur.execute(get_all_records_in_county_specific_year)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2005, 17033, 1, 2, 2, 0, 0, 1, 0, 0)
